In [ ]:
import pandas as pd
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
import datetime as dt
from pandas.tseries.offsets import BDay

yf.pdr_override()

style.use('ggplot')
%matplotlib inline

plt.rcParams['figure.figsize'] = [15, 5]

###########Dates To Use

#First Date of Actuals
startrun = "2016-6-11"

#Last Date of Actuals
endrun = "2018-6-11"

#Forecast start - next business day after the last date of actuals
fcstart = "2018-6-12"

###########Advisable to lower the following number, forecast as few days out as possible, possibly just 3 - 5
forecastdays = 10

In [ ]:
df2 = pd.read_csv('Your csv file here', header=None)
ticklist = df2.copy() #for use later
df2.columns = ['Tickers']
df2 = df2['Tickers'].values.tolist()
ticklist.columns = ['Tickers']
ticklist = ticklist['Tickers'].values.tolist()

In [ ]:
df3=pd.DataFrame()
d = {}

for row in df2:
    d[row] = pd.DataFrame()

for row, df2 in d.items():
    try:
        df3=pdr.get_data_yahoo(row, start = startrun , end = endrun)
        df3['Range'] = (df3['High']-df3['Low'])
        df3['RangeVol'] = ((df3['High']-df3['Low'])/df3['Volume'])*100000
        df3['PrcntChange'] = ((df3['Adj Close']-df3['Open'])/df3['Open'])*100
        d[row]=df3
        
    except:
        pass

In [ ]:
#typing d['GOOGL'], for example will get you the Google price dataframe

In [ ]:
DFwork = pd.concat(d, axis = 1)
DFwork.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn import metrics
from scipy.stats import randint as sp_randint

#for randomized searches for logistic regression and Random Forest
RFRparam_dist = {"bootstrap": [True, False],
                 "max_depth": sp_randint(50, 100),
                 "max_features": sp_randint(2, 7),
                 "min_samples_leaf": sp_randint(2, 10),
                 "min_samples_split": sp_randint(4, 20),
                 "n_jobs": [-1],
                 "n_estimators": sp_randint(1, 500)}

In [ ]:
for ticker in ticklist:
    try:
        DFwork3 = DFwork[ticker]*1
        DFwork3 = DFwork3.drop(columns = ['Close'])
        DFwork3['Dates'] = DFwork3.index
        DFwork3['Days'] = DFwork3['Dates'].dt.dayofweek
        DFwork3 = DFwork3.drop(columns=['Dates'])
        DFwork3['Days'] = DFwork3['Days'].astype(float)
        DFwork4 = DFwork3*1
        DFwork4['Target'] = DFwork4['Adj Close'].shift(-forecastdays)
        DFTarget = DFwork4['Target']
        DFTarget = DFTarget[:-forecastdays]
        DataUsed = DFwork4.drop(columns = ['Target', 'Adj Close'])
        
        scaled_DF = MinMaxScaler().fit_transform(DataUsed)
        scaled_DFedit = pd.DataFrame(scaled_DF, columns=DataUsed.columns)
        ModelVals = scaled_DFedit[:-forecastdays]
        ForecastVals = scaled_DFedit[-forecastdays:]
        
        X_train, X_test, y_train, y_test = train_test_split(ModelVals, DFTarget, test_size=0.3, random_state=3)
        
        randomforestreg = RandomForestRegressor()
        grid_searchrf = RandomizedSearchCV(estimator = randomforestreg, param_distributions = RFRparam_dist, n_iter = 100, cv = 5, n_jobs = -1, verbose = 2)
        grid_searchrf.fit(X_train, y_train)
        best_gridrf = grid_searchrf.best_estimator_
        
        print(best_gridrf)
        best_gridrf.fit(X_train,y_train)
        y_pred = best_gridrf.predict(X_test)
        ForecastPred = best_gridrf.predict(ForecastVals)
        ForecastVals['Pred'] = ForecastPred
        Evaldf=pd.DataFrame({'Actual':y_test, 'Predicted':y_pred, 'Percentage off':((y_pred-y_test)/y_test)*100 })
        best_grid_score = best_gridrf.score(X_test, y_test)
        
        Actualsdf = pd.DataFrame(DFwork3['Adj Close'])
        Actualsdf['Dates'] = Actualsdf.index
        ForecastPreddf=pd.DataFrame(ForecastPred, columns = ['Adj Close'])
        indexfc = pd.bdate_range(fcstart, periods = forecastdays)
        ForecastPreddf['Dates'] = indexfc
        AllDates = Actualsdf.append(ForecastPreddf)
        AllDates['Dates'] =  pd.to_datetime(AllDates['Dates'], format='%Y-%m-%d')
        AllDates.reset_index(drop=True,inplace = True)
        AllDates.set_index('Dates', drop=True,inplace = True)
        
        plt.rcParams['figure.figsize'] = [15, 5]
        plt.subplot(1, 2, 1)
        plt.plot(AllDates[:-forecastdays], color = 'blue', label='Actuals')
        plt.plot(AllDates[-forecastdays:], color = 'red', label='Forecast')
        plt.xlabel("Date")
        plt.ylabel("Price ($)")
        plt.legend()
        plt.title("Price Forecast for " + str(ticker))
        plt.xticks(rotation=75)

        plt.subplot(1, 2, 2)
        plt.plot(AllDates[(int(len(AllDates)/1.10)):-forecastdays], color = 'blue', label='Actuals')
        plt.plot(AllDates[-forecastdays:], color = 'red', label='Forecast')
        plt.xlabel("Date")
        plt.ylabel("Price ($)")
        plt.legend()
        plt.title("Zoomed Price Forecast for " + str(ticker))
        plt.xticks(rotation=75)
        plt.show()
        
        MAE = round(metrics.mean_absolute_error(y_test, y_pred),3)
        MSE = round(metrics.mean_squared_error(y_test, y_pred),3)
        RMSE = round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)),3)
        
        
        print('Mean Absolute Error:', MAE)
        print('Mean Squared Error:', MSE)
        print('Root Mean Squared Error:', RMSE)
    
    except:
        pass